# 2.7 Optimización Básica de Consultas\n\n- Objetivo: patrones simples de mejora: columnas necesarias, filtros eficientes, índices lógicos.\n- Prerrequisitos: consultas de ventas funcionales.\n- Ejercicios: evitar SELECT *, contar sin JOIN, filtros por rango.\n- Reto: diseño de índice compuesto y consulta que lo aproveche.\n- Errores comunes: sobre-indexar; funciones en columnas indexadas; traer columnas no usadas.

In [ ]:
-- Evitar SELECT *: seleccionar columnas necesarias
SELECT venta_id, fecha, cliente_id, producto_id, cantidad
FROM dbo.fact_ventas;

In [ ]:
-- Filtro eficiente: venta por rango de fechas (usa índice si existe por fecha)
SELECT venta_id, fecha, cantidad
FROM dbo.fact_ventas
WHERE fecha BETWEEN '2024-01-01' AND '2024-12-31';

🟢 Ejercicio: Reescribir una consulta que hace JOIN innecesario para solo contar ventas totales.

In [ ]:
-- Antes (innecesario): SELECT COUNT(*) FROM fact_ventas fv JOIN dim_productos p ON fv.producto_id = p.producto_id;
-- Después:
SELECT COUNT(*) AS ventas_totales FROM dbo.fact_ventas;

🟠 Ejercicio: Evaluar impacto de filtrar por producto_id vs sin filtro (estimación conceptual).

🔴 Reto: Diseñar índice compuesto lógico para fact_ventas (fecha, producto_id) y escribir consulta que lo aproveche con filtro de ambos y agregación.

In [ ]:
-- Sugerencia índice (no crear sin análisis):
-- CREATE INDEX IX_fact_ventas_fecha_producto ON dbo.fact_ventas(fecha, producto_id);
-- Consulta que lo aprovecharía:
SELECT producto_id, SUM(cantidad) AS unidades
FROM dbo.fact_ventas
WHERE fecha BETWEEN '2024-01-01' AND '2024-12-31' AND producto_id IN (101,102)
GROUP BY producto_id;

Errores comunes: sobre-indexar; usar funciones sobre columnas indexadas en filtros; traer columnas no usadas (penaliza IO).